In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('processed_data.csv')
df.head()

,Unnamed: 0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,year,month,day,hour,main_category,sub_category,sub_sub_category
0,0,2020-09-24 11:57:06+00:00,view,1996170,2144415922528452715,electronics.telephone,NaN,31.90,1515915625519388267,LJuJVLEjPT,2020,9,24,11,electronics,telephone,NaN
1,1,2020-09-24 11:57:26+00:00,view,139905,2144415926932472027,computers.components.cooler,zalman,17.16,1515915625519380411,tdicluNnRY,2020,9,24,11,computers,components,cooler
2,2,2020-09-24 11:57:27+00:00,view,215454,2144415927158964449,NaN,NaN,9.81,1515915625513238515,4TMArHtXQy,2020,9,24,11,NaN,NaN,NaN
3,3,2020-09-24 11:57:33+00:00,view,635807,2144415923107266682,computers.peripherals.printer,pantum,113.81,1515915625519014356,aGFYrNgC08,2020,9,24,11,computers,peripherals,printer
4,4,2020-09-24 11:57:36+00:00,view,3658723,2144415921169498184,NaN,cameronsino,15.87,1515915625510743344,aa4mmk0kwQ,2020,9,24,11,NaN,NaN,NaN


- 유저 첫 방문일
- 마지막 방문일
- 유저별 누적 세션 수(F)
- 유저별 방문일 수(F)
- 유저별 event_type별 누적 수(F)
- 유저별 누적 소비액(M)
- 유저별 평균 소비액(M)
- 유저별 min&max 

소비액(M)
- 유저별 가장 많이 구입(view, cart)한 브랜드
- 유저별 가장 많이 구입(view, cart)한 카테고리
- 유저별 평균 체류 시간
- 유저방문일로부터 지난 시간

#### 통합코드(각 코드별 결과는 아래에)

In [ ]:
gr=df.groupby('user_id')
last_user_date= gr.agg({'event_time':'max'})
first_user_date= gr.agg({'event_time':'min'})
total_session_amount = gr.agg({'user_session':'count'})
df['event_time'] = pd.to_datetime(df['event_time'])
user_visit_days = gr['event_time'].apply(lambda x: x.dt.date.nunique())
view_count = gr['event_type'].apply(lambda x: (x=='view').sum())
cart_count = gr['event_type'].apply(lambda x: (x=='cart').sum())
purchase_count = gr['event_type'].apply(lambda x: (x=='purchase').sum())
total_purchase_amount = gr.apply(lambda x: x.loc[x['event_type'] == 'purchase', 'price'].sum()).round(3)
average_purchase_amount = (total_purchase_amount/purchase_count).fillna(0).round(3)

## 유저가 마지막 방문(R)

In [3]:
# 유저가 마지막으로 발생시킨 이벤트의 시간(R)

gr=df.groupby('user_id')
last_user_date= gr.agg({'event_time':'max'}) # 1
# last_user_date = gr['event_time'].apply(lambda x:x.max()) # 2
# last_user_date =gr.agg(last_user_date = ('event_time',lambda x: x.max()) # 3
# 다 같은 결과를 반환.
last_user_date.head()

,event_time
user_id,
1515915625353226922,2020-10-29 11:28:35+00:00
1515915625353230067,2020-10-06 06:30:32+00:00
1515915625353230683,2020-12-12 10:33:09+00:00
1515915625353230922,2020-10-02 08:23:40+00:00
1515915625353234047,2021-02-18 09:11:44+00:00


## 유저 첫 방문일(R)

In [5]:
first_user_date= gr.agg({'event_time':'min'})
first_user_date.head()                        

,event_time
user_id,
1515915625353226922,2020-10-29 11:28:35+00:00
1515915625353230067,2020-10-06 06:30:32+00:00
1515915625353230683,2020-11-09 08:52:51+00:00
1515915625353230922,2020-10-02 08:23:40+00:00
1515915625353234047,2020-09-29 16:01:54+00:00


## 유저별 누적 세션 수(F)

In [6]:
# 유저별 누적 세션 수(F)
gr=df.groupby('user_id')
total_session_amount = gr.agg({'user_session':'count'})
total_session_amount.head()

,user_session
user_id,
1515915625353226922,1
1515915625353230067,1
1515915625353230683,13
1515915625353230922,1
1515915625353234047,36


## 유저별 방문일 수(F)

In [32]:
# 유저별 방문일 수(F)
df['event_time'] = pd.to_datetime(df['event_time'])
user_visit_days = gr['event_time'].apply(lambda x: x.dt.date.nunique())
user_visit_days.head() 

user_id
1515915625353226922     1
1515915625353230067     1
1515915625353230683     3
1515915625353230922     1
1515915625353234047    10
Name: event_time, dtype: int64

## 유저별 event_type별 누적 수(F)

In [8]:
# 유저별 event_type별 누적 수(F)
view_count = gr['event_type'].apply(lambda x: (x=='view').sum())
cart_count = gr['event_type'].apply(lambda x: (x=='cart').sum())
purchase_count = gr['event_type'].apply(lambda x: (x=='purchase').sum())
purchase_count.head() 

user_id
1515915625353226922    0
1515915625353230067    0
1515915625353230683    0
1515915625353230922    0
1515915625353234047    0
Name: event_type, dtype: int64

In [14]:
(purchase_count>=1).sum() # 구매가 1개 이상인 21304개 정도 있음

21304

In [15]:
(purchase_count==0).sum()

385979

## 유저별 누적 소비액(M)

In [33]:
total_purchase_amount = gr.apply(lambda x: x.loc[x['event_type'] == 'purchase', 'price'].sum()).round(3)
total_purchase_amount.head()

user_id
1515915625353226922    0.0
1515915625353230067    0.0
1515915625353230683    0.0
1515915625353230922    0.0
1515915625353234047    0.0
dtype: float64

## 유저별 평균 소비액(M)

In [34]:
average_purchase_amount = (total_purchase_amount/purchase_count).fillna(0).round(3)
print(average_purchase_amount.head(),'\n결측치가 아닌 시리즈 값의 합은: ',
average_purchase_amount[~(average_purchase_amount).isnull()].sum())

user_id
1515915625353226922    0.0
1515915625353230067    0.0
1515915625353230683    0.0
1515915625353230922    0.0
1515915625353234047    0.0
dtype: float64 
결측치가 아닌 시리즈 값의 합은:  2914392.2590000005


In [41]:
combined_df = pd.concat([
    last_user_date,
    first_user_date,
    total_session_amount,
    user_visit_days,
    view_count,
    cart_count,
    purchase_count,
    total_purchase_amount,
    average_purchase_amount
], axis=1).reset_index()

In [42]:
combined_df

,index,user_id,last_user_date,user_id,first_user_date,user_id,total_session_amount,user_id,user_visit_days,user_id,view_count,user_id,cart_count,user_id,purchase_count,user_id,total_purchase_amount,user_id,average_purchase_amount
0,0,1515915625353226922,2020-10-29 11:28:35+00:00,1515915625353226922,2020-10-29 11:28:35+00:00,1515915625353226922,1,1515915625353226922,1,1515915625353226922,1,1515915625353226922,0,1515915625353226922,0,1515915625353226922,0.0,0,0.0
1,1,1515915625353230067,2020-10-06 06:30:32+00:00,1515915625353230067,2020-10-06 06:30:32+00:00,1515915625353230067,1,1515915625353230067,1,1515915625353230067,1,1515915625353230067,0,1515915625353230067,0,1515915625353230067,0.0,1,0.0
2,2,1515915625353230683,2020-12-12 10:33:09+00:00,1515915625353230683,2020-11-09 08:52:51+00:00,1515915625353230683,4,1515915625353230683,3,1515915625353230683,13,1515915625353230683,0,1515915625353230683,0,1515915625353230683,0.0,2,0.0
3,3,1515915625353230922,2020-10-02 08:23:40+00:00,1515915625353230922,2020-10-02 08:23:40+00:00,1515915625353230922,1,1515915625353230922,1,1515915625353230922,1,1515915625353230922,0,1515915625353230922,0,1515915625353230922,0.0,3,0.0
4,4,1515915625353234047,2021-02-18 09:11:44+00:00,1515915625353234047,2020-09-29 16:01:54+00:00,1515915625353234047,1,1515915625353234047,10,1515915625353234047,36,1515915625353234047,0,1515915625353234047,0,1515915625353234047,0.0,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407278,407278,1515915625611023671,2021-02-28 23:54:18+00:00,1515915625611023671,2021-02-28 23:54:18+00:00,1515915625611023671,1,1515915625611023671,1,1515915625611023671,1,1515915625611023671,0,1515915625611023671,0,1515915625611023671,0.0,407278,0.0
407279,407279,1515915625611023730,2021-02-28 23:55:01+00:00,1515915625611023730,2021-02-28 23:55:01+00:00,1515915625611023730,1,1515915625611023730,1,1515915625611023730,1,1515915625611023730,0,1515915625611023730,0,1515915625611023730,0.0,407279,0.0
407280,407280,1515915625611024014,2021-02-28 23:58:05+00:00,1515915625611024014,2021-02-28 23:58:05+00:00,1515915625611024014,1,1515915625611024014,1,1515915625611024014,1,1515915625611024014,0,1515915625611024014,0,1515915625611024014,0.0,407280,0.0
407281,407281,1515915625611024020,2021-02-28 23:58:09+00:00,1515915625611024020,2021-02-28 23:58:09+00:00,1515915625611024020,1,1515915625611024020,1,1515915625611024020,1,1515915625611024020,0,1515915625611024020,0,1515915625611024020,0.0,407281,0.0


## 유저별 min&max 소비액(M)

In [22]:
# 모든 유저 리스트
all_users = df['user_id'].unique()
purchase_df = df[df['event_type'] == 'purchase']

max_purchase_amount = purchase_df.groupby('user_id')['price'].max().reset_index(name='max_amount')
min_purchase_amount = purchase_df.groupby('user_id')['price'].min().reset_index(name='min_amount')

# 모든 유저를 포함한 데이터프레임 생성
max_amount_per_user = pd.DataFrame(all_users, columns=['user_id']).merge(max_purchase_amount, on='user_id', how='left')
max_amount_per_user['max_amount'] = max_amount_per_user['max_amount'].fillna(0)
min_amount_per_user = pd.DataFrame(all_users, columns=['user_id']).merge(min_purchase_amount, on='user_id', how='left')
min_amount_per_user['min_amount'] = min_amount_per_user['min_amount'].fillna(0)


amount_per_user = max_amount_per_user.merge(min_amount_per_user, on='user_id')
amount_per_user.head()

,user_id,max_amount,min_amount
0,1515915625519388267,0.0,0.0
1,1515915625519380411,0.0,0.0
2,1515915625513238515,0.0,0.0
3,1515915625519014356,0.0,0.0
4,1515915625510743344,0.0,0.0


## 통합
```
- 유저 첫 방문일
- 마지막 방문일
- 유저별 누적 세션 수(F)
- 유저별 방문일 수(F)
- 유저별 event_type별 누적 수(F)
- 유저별 누적 소비액(M)
- 유저별 평균 소비액(M)
- 유저별 min&max 소비액(M)
```

In [43]:

gr = df.groupby('user_id')
last_user_date = gr.agg({'event_time': 'max'}).rename(columns={'event_time': 'last_user_date'})
first_user_date = gr.agg({'event_time': 'min'}).rename(columns={'event_time': 'first_user_date'})
total_session_amount = gr.agg({'user_session': 'count'}).rename(columns={'user_session': 'total_session_amount'})

df['event_time'] = pd.to_datetime(df['event_time'])
user_visit_days = gr['event_time'].apply(lambda x: x.dt.date.nunique()).rename('user_visit_days')
view_count = gr['event_type'].apply(lambda x: (x=='view').sum()).rename('view_count')
cart_count = gr['event_type'].apply(lambda x: (x=='cart').sum()).rename('cart_count')
purchase_count = gr['event_type'].apply(lambda x: (x=='purchase').sum()).rename('purchase_count')
total_purchase_amount = gr.apply(lambda x: x.loc[x['event_type'] == 'purchase', 'price'].sum()).round(3).rename('total_purchase_amount')
average_purchase_amount = (total_purchase_amount/purchase_count).fillna(0).round(3).rename('average_purchase_amount')


combined_df = pd.concat([
    last_user_date,
    first_user_date,
    total_session_amount,
    user_visit_days,
    view_count,
    cart_count,
    purchase_count,
    total_purchase_amount,
    average_purchase_amount
], axis=1).reset_index()


combined_df


,user_id,last_user_date,first_user_date,total_session_amount,user_visit_days,view_count,cart_count,purchase_count,total_purchase_amount,average_purchase_amount
0,1515915625353226922,2020-10-29 11:28:35+00:00,2020-10-29 11:28:35+00:00,1,1,1,0,0,0.0,0.0
1,1515915625353230067,2020-10-06 06:30:32+00:00,2020-10-06 06:30:32+00:00,1,1,1,0,0,0.0,0.0
2,1515915625353230683,2020-12-12 10:33:09+00:00,2020-11-09 08:52:51+00:00,13,3,13,0,0,0.0,0.0
3,1515915625353230922,2020-10-02 08:23:40+00:00,2020-10-02 08:23:40+00:00,1,1,1,0,0,0.0,0.0
4,1515915625353234047,2021-02-18 09:11:44+00:00,2020-09-29 16:01:54+00:00,36,10,36,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
407278,1515915625611023671,2021-02-28 23:54:18+00:00,2021-02-28 23:54:18+00:00,1,1,1,0,0,0.0,0.0
407279,1515915625611023730,2021-02-28 23:55:01+00:00,2021-02-28 23:55:01+00:00,1,1,1,0,0,0.0,0.0
407280,1515915625611024014,2021-02-28 23:58:05+00:00,2021-02-28 23:58:05+00:00,1,1,1,0,0,0.0,0.0
407281,1515915625611024020,2021-02-28 23:58:09+00:00,2021-02-28 23:58:09+00:00,1,1,1,0,0,0.0,0.0


In [45]:
combined=pd.merge(amount_per_user,combined_df,on='user_id',how='inner')

In [46]:
combined

,user_id,max_amount,min_amount,last_user_date,first_user_date,total_session_amount,user_visit_days,view_count,cart_count,purchase_count,total_purchase_amount,average_purchase_amount
0,1515915625519388267,0.0,0.0,2020-09-24 11:57:06+00:00,2020-09-24 11:57:06+00:00,1,1,1,0,0,0.0,0.0
1,1515915625519380411,0.0,0.0,2020-09-24 12:19:57+00:00,2020-09-24 11:57:26+00:00,3,1,3,0,0,0.0,0.0
2,1515915625513238515,0.0,0.0,2020-09-27 13:01:07+00:00,2020-09-24 11:57:27+00:00,2,2,2,0,0,0.0,0.0
3,1515915625519014356,0.0,0.0,2020-09-24 11:57:33+00:00,2020-09-24 11:57:33+00:00,1,1,1,0,0,0.0,0.0
4,1515915625510743344,0.0,0.0,2020-09-24 11:57:36+00:00,2020-09-24 11:57:36+00:00,1,1,1,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
407278,1515915625611023671,0.0,0.0,2021-02-28 23:54:18+00:00,2021-02-28 23:54:18+00:00,1,1,1,0,0,0.0,0.0
407279,1515915625611023730,0.0,0.0,2021-02-28 23:55:01+00:00,2021-02-28 23:55:01+00:00,1,1,1,0,0,0.0,0.0
407280,1515915625611024014,0.0,0.0,2021-02-28 23:58:05+00:00,2021-02-28 23:58:05+00:00,1,1,1,0,0,0.0,0.0
407281,1515915625611024020,0.0,0.0,2021-02-28 23:58:09+00:00,2021-02-28 23:58:09+00:00,1,1,1,0,0,0.0,0.0


In [47]:
combined.to_csv('combinded_user_df_75%.csv')